In [ ]:
# Instalación de RAPIDS
!pip install --upgrade --quiet "cupy-cuda12x"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 MB 8.3 MB/s eta 0:00:00


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time
import gc

# Configuración general
DATA_DIR = "/img/ImageNet/ILSVRC2012"
BATCH_SIZE = 500
NUM_WORKERS = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Transformación estándar de ImageNet con salida 224x224
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Carga del dataset
imagenet_dataset = datasets.ImageNet(root=DATA_DIR, split='train', transform=transform)
data_loader = DataLoader(imagenet_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

def process_batch(images):
    images = images.to(DEVICE, non_blocking=True)
    _ = images.mean()
    del images
    torch.cuda.empty_cache()
    gc.collect()

def main():
    total_images = len(imagenet_dataset)
    print(f"Procesando {total_images} imágenes en GPU por batches de {BATCH_SIZE}...")

    start_gpu = time.time()

    for i, (images, _) in enumerate(data_loader):
        process_batch(images)
        if (i + 1) % 20 == 0:
            print(f"Batch {i + 1} procesado")

    torch.cuda.synchronize()
    end_gpu = time.time()

    tiempo_gpu = end_gpu - start_gpu
    print(f"\nTiempo total GPU: {tiempo_gpu:.2f} segundos")

if __name__ == "__main__":
    main()


Procesando 1300000 imágenes en GPU por batches de 500...
Batch 20/2600 procesado
Batch 40/2600 procesado
Batch 60/2600 procesado
Batch 80/2600 procesado
Batch 100/2600 procesado
Batch 120/2600 procesado
Batch 140/2600 procesado
Batch 160/2600 procesado
Batch 180/2600 procesado
Batch 200/2600 procesado
Batch 220/2600 procesado
Batch 240/2600 procesado
Batch 260/2600 procesado
Batch 280/2600 procesado
Batch 300/2600 procesado
Batch 320/2600 procesado
Batch 340/2600 procesado
Batch 360/2600 procesado
Batch 380/2600 procesado
Batch 400/2600 procesado
Batch 420/2600 procesado
Batch 440/2600 procesado
Batch 460/2600 procesado
Batch 480/2600 procesado
Batch 500/2600 procesado
Batch 520/2600 procesado
Batch 540/2600 procesado
Batch 560/2600 procesado
Batch 580/2600 procesado
Batch 600/2600 procesado
Batch 620/2600 procesado
Batch 640/2600 procesado
Batch 660/2600 procesado
Batch 680/2600 procesado
Batch 700/2600 procesado
Batch 720/2600 procesado
Batch 740/2600 procesado
Batch 760/2600 procesa

In [ ]:
import numpy as np
import torch
import time
import gc
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Configuración general
DATA_DIR = "/img/ImageNet/ILSVRC2012"
BATCH_SIZE_GPU = 500
BATCH_SIZE_CPU = 1
NUM_WORKERS = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Transformación
imagenet_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Dataset y dataloaders
imagenet_dataset = datasets.ImageNet(root=DATA_DIR, split='train', transform=imagenet_transform)

# DataLoader para CPU (batch de 1 imagen por iteración)
cpu_loader = DataLoader(imagenet_dataset, batch_size=BATCH_SIZE_CPU, shuffle=False, num_workers=0)

# DataLoader para GPU (batch grande por eficiencia)
gpu_loader = DataLoader(imagenet_dataset, batch_size=BATCH_SIZE_GPU, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

#  CPU VERSION

def cpu_preprocess():
    print("\n== Procesando en CPU ==")
    start_cpu = time.time()

    for i, (img, _) in enumerate(cpu_loader):
        img_np = img.squeeze(0).numpy()
        _ = img_np.mean()
        if (i + 1) % 200000 == 0:
            print(f"{i + 1}/{len(imagenet_dataset)} imágenes procesadas (CPU)")

    end_cpu = time.time()
    tiempo_cpu = end_cpu - start_cpu
    print(f"Tiempo CPU: {tiempo_cpu:.2f} segundos")
    return tiempo_cpu


def gpu_process_batch(images):
    images = images.to(DEVICE, non_blocking=True)
    _ = images.mean()
    del images
    torch.cuda.empty_cache()
    gc.collect()

def gpu_preprocess():
    print("\n== Procesando en GPU ==")
    start_gpu = time.time()

    for i, (images, _) in enumerate(gpu_loader):
        gpu_process_batch(images)
        if (i + 1) % 20 == 0:
            print(f"Batch {i + 1} procesado")

    torch.cuda.synchronize()
    end_gpu = time.time()
    tiempo_gpu = end_gpu - start_gpu
    print(f"Tiempo GPU: {tiempo_gpu:.2f} segundos")
    return tiempo_gpu



def main():
    tiempo_cpu = cpu_preprocess()
    tiempo_gpu = gpu_preprocess()

    print("\n== Benchmark final ==")
    print(f"Tiempo CPU: {tiempo_cpu:.2f} s")
    print(f"Tiempo GPU: {tiempo_gpu:.2f} s")
    print(f"Speedup GPU vs CPU: {tiempo_cpu / tiempo_gpu:.2f}x")

if __name__ == "__main__":
    main()



== Procesando en CPU ==
200000/1300000 imágenes procesadas (CPU)
400000/1300000 imágenes procesadas (CPU)
600000/1300000 imágenes procesadas (CPU)
800000/1300000 imágenes procesadas (CPU)
1000000/1300000 imágenes procesadas (CPU)
1200000/1300000 imágenes procesadas (CPU)
Tiempo CPU: 809.11 segundos

== Procesando en GPU ==
Batch 20/2600 procesado
Batch 40/2600 procesado
Batch 60/2600 procesado
Batch 80/2600 procesado
Batch 100/2600 procesado
Batch 120/2600 procesado
Batch 140/2600 procesado
Batch 160/2600 procesado
Batch 180/2600 procesado
Batch 200/2600 procesado
Batch 220/2600 procesado
Batch 240/2600 procesado
Batch 260/2600 procesado
Batch 280/2600 procesado
Batch 300/2600 procesado
Batch 320/2600 procesado
Batch 340/2600 procesado
Batch 360/2600 procesado
Batch 380/2600 procesado
Batch 400/2600 procesado
Batch 420/2600 procesado
Batch 440/2600 procesado
Batch 460/2600 procesado
Batch 480/2600 procesado
Batch 500/2600 procesado
Batch 520/2600 procesado
Batch 540/2600 procesado
Bat